In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed
from model.load_data import train_test_split, train_test_split_LSTM

from model.scoring_metrics import get_windiff, get_pk, get_k_kappa

batch_size = 64

# I optimize on this, I think?
LSTM_units = 20

features = ['pause','speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']

In [9]:
!pip install tensorflow-addons
!pip install typeguard

In [3]:
import keras.backend as K

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

import warnings
from typing import Optional
from typeguard import typechecked
from tensorflow_addons.utils.types import Number

class WeightedKappaLoss(tf.keras.losses.Loss):
    @typechecked
    def __init__(
            self,
            num_classes: int,
            weightage: Optional[str] = "quadratic",
            name: Optional[str] = "cohen_kappa_loss",
            epsilon: Optional[Number] = 1e-6,
            dtype: Optional[tf.DType] = tf.float32,
            reduction: str = tf.keras.losses.Reduction.NONE,
    ):
        super().__init__(name=name, reduction=reduction)
        warnings.warn(
            "The data type for `WeightedKappaLoss` defaults to "
            "`tf.keras.backend.floatx()`."
            "The argument `dtype` will be removed in Addons `0.12`.",
            DeprecationWarning,
        )
        if weightage not in ("linear", "quadratic"):
            raise ValueError("Unknown kappa weighting type.")

        self.weightage = weightage
        self.num_classes = num_classes
        self.epsilon = epsilon or tf.keras.backend.epsilon()
        label_vec = tf.range(num_classes, dtype=tf.keras.backend.floatx())
        self.row_label_vec = tf.reshape(label_vec, [1, num_classes])
        self.col_label_vec = tf.reshape(label_vec, [num_classes, 1])
        col_mat = tf.tile(self.col_label_vec, [1, num_classes])
        row_mat = tf.tile(self.row_label_vec, [num_classes, 1])
        if weightage == "linear":
            self.weight_mat = tf.abs(col_mat - row_mat)
        else:
            self.weight_mat = (col_mat - row_mat) ** 2

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, dtype=self.col_label_vec.dtype)
        y_pred = tf.cast(y_pred, dtype=self.weight_mat.dtype)
        batch_size = tf.shape(y_true)[0]
        cat_labels = tf.matmul(y_true, self.col_label_vec)
        cat_label_mat = tf.tile(cat_labels, [1, self.num_classes])
        row_label_mat = tf.tile(self.row_label_vec, [batch_size, 1])
        if self.weightage == "linear":
            weight = tf.abs(cat_label_mat - row_label_mat)
        else:
            weight = (cat_label_mat - row_label_mat) ** 2
        numerator = tf.reduce_sum(weight * y_pred)
        label_dist = tf.reduce_sum(y_true, axis=0, keepdims=True)
        pred_dist = tf.reduce_sum(y_pred, axis=0, keepdims=True)
        w_pred_dist = tf.matmul(self.weight_mat, pred_dist, transpose_b=True)
        denominator = tf.reduce_sum(tf.matmul(label_dist, w_pred_dist))
        denominator /= tf.cast(batch_size, dtype=denominator.dtype)
        loss = tf.math.divide_no_nan(numerator, denominator)
        return tf.math.log(loss + self.epsilon)

    def get_config(self):
        config = {
            "num_classes": self.num_classes,
            "weightage": self.weightage,
            "epsilon": self.epsilon,
        }
        base_config = super().get_config()
        return {**base_config, **config}

In [4]:
#load the data
from model_trainer_and_tester import read_in_dataset_lstm, test_set_evaluate_multiple_lstm

shifts = [-2, -1, 0, 1, 2]

n_timesteps = len(shifts)
feature_count = len(features)

X_train, Y_train = read_in_dataset_lstm(features, shifts, to_read='train')
X_test, Y_test = read_in_dataset_lstm(features, shifts, to_read='test')
#train_test_split_LSTM(datasets, results_merged_path, n_timesteps, split=train_ratio)

print(X_train.shape)
print(Y_train.shape)

# So, the model should be tanh activations and sigmoid outpiut layers...
# I'm going to guess both layers should be blstm... but that's it pretty much I think?

(47450, 5, 5)
(47450, 5, 1)


In [18]:
# I'm going to do sample weight here. For now I'll just set the weight to be... half of the number of weird samples?

sample_weight = np.ones(shape=(len(Y_train),))
# I'm gonna increase the weight by the number of weird examples that there are
# It sums along the 2D squares to find where there's a 1, and then does a sum of times there's a 1
new_weight = np.sum(Y_train, axis=1).sum()/2.0

# Have to do a flatten() inside because of weird numpy stuff with a length 1 dimension
sample_weight[(np.sum(Y_train, axis=1) >= 1).flatten()] = new_weight

sample_weight

array([1., 1., 1., ..., 1., 1., 1.])

In [22]:
from tensorflow import keras

# TODO: Understand how the hell to build a keras model
model = Sequential()
# For the input number of units, I'll assume that number of timesteps * features is a good enough value
model.add(Bidirectional(LSTM(n_timesteps*feature_count, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))
model.add(Bidirectional(LSTM(LSTM_units, activation='sigmoid', return_sequences=True, dropout=0.3)))
# This last time distributed is super important, it follows the output structure of the paper I've been following closely
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='RMSprop',
              metrics=[keras.metrics.Precision(), tf.keras.metrics.Recall()], weighted_metrics=[])

# train the model
history=model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=3,
                  #class_weight= {0:1, 1:10},
                  # sample_weight_mode='temporal',
                  sample_weight=sample_weight,
                  validation_split=0.1,
                  verbose=1
                  )

hist_results_dict = {}
acc_results_dict = {}

Epoch 1/3
668/668 [==============================] - 19s 16ms/step - loss: 25.9344 - precision_2: 0.0396 - recall_2: 0.0124 - val_loss: 21.6583 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/3
668/668 [==============================] - 8s 12ms/step - loss: 24.7127 - precision_2: 0.0766 - recall_2: 0.0530 - val_loss: 20.7949 - val_precision_2: 0.2059 - val_recall_2: 0.0365
Epoch 3/3
668/668 [==============================] - 8s 13ms/step - loss: 24.4539 - precision_2: 0.0629 - recall_2: 0.0669 - val_loss: 20.2637 - val_precision_2: 0.1064 - val_recall_2: 0.0521


In [20]:
results = test_set_evaluate_multiple_lstm(model, features, shifts)
#temp_y = model.predict(X_train)
results

86/86 [==============================] - 0s 3ms/step


,Pk,K-k,Windiff
0,0.554391,-0.202583,0.586282
1,0.380090,-0.161110,0.380090
2,0.321965,0.328838,0.360164
3,0.581089,-0.177916,0.623496
4,0.466907,0.019918,0.544525
5,0.234568,0.406358,0.360624
6,0.567990,-0.256381,0.572804
7,0.289179,0.324772,0.309701
8,0.353620,0.241495,0.368311
9,0.534402,0.000000,0.534402


In [64]:
for num_layers in range(1, 5):
    # It's 31 per step, because it allows for there to be 16 trials, and for me to not go crazy
    for hidden_units in range(16, 513, 31):
        #print("Starting model: " + str((num_layers, hidden_units)))
        model = Sequential()
        # For the input number of units, I'll assume that number of timesteps * features is a good enough value
        for _ in range(num_layers):
            model.add(Bidirectional(LSTM(hidden_units, activation='tanh', return_sequences=True, dropout=0.3), input_shape=(n_timesteps, feature_count)))
        model.add(Bidirectional(LSTM(hidden_units, activation='sigmoid', return_sequences=True, dropout=0.3)))
        # This last time distributed is super important, it follows the output structure of the paper I've been following closely
        model.add(TimeDistributed(Dense(1, activation='sigmoid')))
        model.compile(loss='binary_crossentropy', optimizer='RMSprop',
                      metrics=[keras.metrics.Precision(), tf.keras.metrics.Recall()], weighted_metrics=[])

        # train the model
        history=model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=3,
                  #class_weight= {0:1, 1:10},
                  # sample_weight_mode='temporal',
                  sample_weight=sample_weight,
                  validation_split=0.1,
                  verbose=1
                  )

        print("Finished model: " + str((num_layers, hidden_units)) + " with results: " + str(history.history['val_acc'][-1]))

        hist_results_dict[(num_layers, hidden_units)] = history
        # It's -1, to take the last accuracy measure obtained
        acc_results_dict[(num_layers, hidden_units)] = history.history['val_acc'][-1]

Starting model: (1, 16)
Finished model: (1, 16)
Starting model: (1, 47)
Finished model: (1, 47)
Starting model: (1, 78)
Finished model: (1, 78)
Starting model: (1, 109)
Finished model: (1, 109)
Starting model: (1, 140)
Finished model: (1, 140)
Starting model: (1, 171)
Finished model: (1, 171)
Starting model: (1, 202)
Finished model: (1, 202)
Starting model: (1, 233)
Finished model: (1, 233)
Starting model: (1, 264)
Finished model: (1, 264)
Starting model: (1, 295)
Finished model: (1, 295)
Starting model: (1, 326)
Finished model: (1, 326)
Starting model: (1, 357)
Finished model: (1, 357)
Starting model: (1, 388)
Finished model: (1, 388)
Starting model: (1, 419)
Finished model: (1, 419)
Starting model: (1, 450)
Finished model: (1, 450)
Starting model: (1, 481)
Finished model: (1, 481)
Starting model: (1, 512)
Finished model: (1, 512)
Starting model: (2, 16)
Finished model: (2, 16)
Starting model: (2, 47)
Finished model: (2, 47)
Starting model: (2, 78)
Finished model: (2, 78)
Starting mod

KeyboardInterrupt: 

In [65]:
acc_results_dict

{(1, 16): 0.9919072985649109,
 (1, 47): 0.9919072985649109,
 (1, 78): 0.9919072985649109,
 (1, 109): 0.9919072985649109,
 (1, 140): 0.9919072985649109,
 (1, 171): 0.9919072985649109,
 (1, 202): 0.9919072985649109,
 (1, 233): 0.9919072985649109,
 (1, 264): 0.9919072985649109,
 (1, 295): 0.9919072985649109,
 (1, 326): 0.9919072985649109,
 (1, 357): 0.9919072985649109,
 (1, 388): 0.9919072985649109,
 (1, 419): 0.9919072985649109,
 (1, 450): 0.9919072985649109,
 (1, 481): 0.9919072985649109,
 (1, 512): 0.9919072985649109,
 (2, 16): 0.9919072985649109,
 (2, 47): 0.9919072985649109,
 (2, 78): 0.9919072985649109,
 (2, 109): 0.9919072985649109,
 (2, 140): 0.9919072985649109,
 (2, 171): 0.9919072985649109,
 (2, 202): 0.9919072985649109,
 (2, 233): 0.9919072985649109,
 (2, 264): 0.9919072985649109,
 (2, 295): 0.9919072985649109,
 (2, 326): 0.9919072985649109,
 (2, 357): 0.9919072985649109,
 (2, 388): 0.9919072985649109,
 (2, 419): 0.9919072985649109,
 (2, 450): 0.9919072985649109,
 (2, 481): 0.9

In [31]:
results = test_set_evaluate_multiple_lstm(model, features, shifts)
#temp_y = model.predict(X_train)
results

86/86 [==============================] - 0s 3ms/step


,Pk,K-k,Windiff
0,0.372652,0.0,0.372652
1,0.095525,0.0,0.095525
2,0.470668,0.0,0.470668
3,0.477364,0.0,0.477364
4,0.439832,0.0,0.439832
5,0.283951,0.0,0.283951
6,0.397112,0.0,0.397112
7,0.380597,0.0,0.380597
8,0.441763,0.0,0.441763
9,0.534402,0.0,0.534402


In [32]:
results.mean()

Pk         0.404572
K-k        0.000000
Windiff    0.404572
dtype: float64

In [18]:
temp_y = model.predict(X_train)
temp_y

1483/1483 [==============================] - 5s 3ms/step


array([[[0.00627282],
        [0.01584293],
        [0.01336911],
        [0.02408313],
        [0.01425999]],

       [[0.00942823],
        [0.00914664],
        [0.01536419],
        [0.0124401 ],
        [0.00516671]],

       [[0.00379193],
        [0.00422932],
        [0.00482995],
        [0.00295994],
        [0.00844802]],

       ...,

       [[0.01065987],
        [0.02366466],
        [0.00473628],
        [0.0050675 ],
        [0.00676939]],

       [[0.02293292],
        [0.00628696],
        [0.00566995],
        [0.00733351],
        [0.00526068]],

       [[0.00610961],
        [0.00701646],
        [0.0087015 ],
        [0.00562242],
        [0.00662829]]], dtype=float32)

It's working which is good. It is giving garbage results though.